In [1]:
import random
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.model_selection import KFold

from Python_Scripts import OrderedCategorySystem as OCS
from Python_Scripts import generate_plots as plots
from Python_Scripts import order_analyses as analyses
from Python_Scripts import RationalCategorySystem as RCS

In [2]:
F =  [12, 13, 15, 14, 16, 18, 17, 19, 20]
B =  [20, 19, 17, 18, 16, 14, 15, 13, 12]
M1 =  [16, 17, 15, 18, 14, 19, 13, 20, 12]
M2 = [16, 15, 17, 14, 18, 13, 19, 12, 20]

NEW = [i for i in range(9, 24)]
ALL = NEW + [1, 3, 29, 31]

SHIFT = 3
DISTRACTORS = [1, 3, 29, 31]

ITEMS = ['I09', 'I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'I16', 'I17', 'I18', 'I19', 'I20', 'I21', 'I22', 'I23']

LEFT = ITEMS[:9]
CENTRE = ITEMS[3:12]
RIGHT = ITEMS[6:]

LOCS = [('L', LEFT),
        ('C', CENTRE), 
        ('R', RIGHT)]
ORDERS = [('f', 0, [1, 2, 3, 4, 5, 6, 7, 8]),
          ('m', 4, [0, 1, 2, 3, 5, 6, 7, 8]),
          ('b', 8, [0, 1, 2, 3, 4, 5, 6, 7])]

item_space = [i for i in range(1, 32)]

### Load Participant Data

In [3]:
allParticipants = pd.read_csv('Results/participant_data.csv')
allParticipants =  allParticipants[(allParticipants['ATTEMPTS'] < 3) & (allParticipants['TOTAL_ERRORS'] < 4) & (allParticipants['POOL'] == 'prolific2')]
participants = allParticipants['P_ID'].tolist()

participant_df = pd.read_csv('Results/trial_data.csv')    
participant_df = participant_df[participant_df['P_ID'].isin(participants)]

cat_assigns = ITEMS + ['I01', 'I03', 'I29', 'I31']
others = participant_df.columns.difference(cat_assigns)
sequence_df = pd.read_csv('Results/sequence_data.csv')

trial_df = (
  participant_df[others]
    .assign(ITEMS = participant_df[cat_assigns].agg(
            lambda row: {k: v for k, v in row.items() if not pd.isna(v)},
            axis=1
    )
  )
)
seq_df = (
  sequence_df[['P_ID', 'DEPTH', 'LOC', 'ORDER', 'STIMULI']]
  .assign(SEQUENCE = sequence_df[[f't{i+1:02}' for i in range(13)]].agg(
            lambda row: {k: v for k, v in row.items()},
            axis=1
    )   
  )
)

data_df = trial_df.merge(seq_df, on=['P_ID', 'DEPTH', 'LOC', 'ORDER', 'STIMULI'], how='inner')
participant_trials = list(data_df.to_dict('index').values())

In [4]:

def format_rcm_data(trial_data):
    rcmData = {t['P_ID']:[] for t in trial_data}
    for t in trial_data:
        item_assignments = {int(key[1:]): val for key, val in t['ITEMS'].items()}
        sequence = {key: int(val) for key, val in t['SEQUENCE'].items()}
        rcmData[t['P_ID']].append({'CHOICES': item_assignments, 'SEQUENCE': sequence, 'DEPTH': t['DEPTH'], 'LOC': t['LOC'], 'ORDER': t['ORDER'], 'STIMULI': t['STIMULI']})
    return rcmData

def get_trial_loglike(model, choices, order, item_space, alpha=0.0, uniform=None):
    log_like = 0
    for t in range(len(order)):
        it = order[f't{t+1:02}']
        choice = choices[it]
        item_rep = item_space[it-1]
        if 0.0 < alpha < 1.0 and uniform is not None:
            prob_dist, choice_idx = model.get_item_likelihood(item_rep, choice)
            mixture = np.logaddexp(np.log(1 - alpha) + prob_dist[choice_idx], np.log(alpha) + np.log(uniform))
        elif alpha == 1.0 and uniform is not None:
            mixture = np.log(uniform)
        else:
            prob_dist, choice_idx = model.get_item_likelihood(item_rep, choice)
            mixture =  prob_dist[choice_idx]
        log_like += mixture
    return log_like

def get_participant_loglike_rcm(data, item_space, c=0.5, alpha=0.0, diff3=False):
    total_ll = 0.0
    existing_items = [item_space[i] for i in [1, 2, 3, 5, 6, 7, 23, 24, 25, 27, 28, 29]]
    mu_0 = [np.mean(item_space)]
    var_0 = [np.var(item_space)]
    for trial in data:
        d = trial['DEPTH']
        order = trial['SEQUENCE']
        choices = trial['CHOICES']
        if diff3 and d == 3:
            syst = ['L1', 'L1', 'L1', 'L2', 'L2', 'L2', 'R2', 'R2', 'R2', 'R1', 'R1', 'R1']
            max_new = 3
            uniform = 1/7
        else:
            syst = ['L',  'L',  'L',  'L',  'L',  'L',  'R',  'R',  'R',  'R',  'R',  'R']
            max_new = 1
            uniform = 1/3
            if d == 3:
                choices = {key: val[0] for key, val in choices.items()}
        rcm = RCS.RationalModel(c, mu_0, var_0,  np.array([1.0]),  np.array([1.0]), partition=syst, stimuli=existing_items, max_new_clusters=max_new)
        total_ll += get_trial_loglike(rcm, choices, order, item_space, alpha, uniform)
    return total_ll

def get_total_log_like_rcm(trial_data, item_space, c=0.5, alpha=0.0, diff3=False):
    total_ll = 0.0
    for _, p_data in trial_data.items():
        total_ll += get_participant_loglike_rcm(p_data, item_space, c, alpha, diff3)
    return total_ll

def cross_val_branch(data, temps, alphas, outer_k=5, inner_k=5):
    participant_keys = list(data.keys())
    outer_kf = KFold(n_splits=outer_k, shuffle=True, random_state=42)
    
    outer_val_lls = []
    best_hyperparams_per_fold = []

    for outer_train_idx, outer_test_idx in outer_kf.split(participant_keys):
        outer_train_keys = [participant_keys[i] for i in outer_train_idx]
        outer_test_keys = [participant_keys[i] for i in outer_test_idx]

        outer_train_data = {k: data[k] for k in outer_train_keys}
        outer_test_data = {k: data[k] for k in outer_test_keys}

        # --- INNER CV: select best (t, alpha) ---
        inner_kf = KFold(n_splits=inner_k, shuffle=True, random_state=42)
        mean_val_ll_per_hyper = {}
        for t in temps:
            for alpha in alphas:
                val_lls_inner = []
                skip_alpha = False

                for inner_train_idx, inner_val_idx in inner_kf.split(outer_train_keys):
                    inner_train_keys = [outer_train_keys[i] for i in inner_train_idx]
                    inner_val_keys = [outer_train_keys[i] for i in inner_val_idx]

                    inner_train_data = {k: data[k] for k in inner_train_keys}
                    inner_val_data = {k: data[k] for k in inner_val_keys}

                    # Reset prev_ll per inner fold
                    prev_ll_train = None  

                    ll_train = OCS.get_total_log_like(inner_train_data, t, alpha)

                    if prev_ll_train is not None and ll_train < prev_ll_train:
                        skip_alpha = True
                        break

                    prev_ll_train = ll_train

                    # Compute validation LL
                    ll_val = OCS.get_total_log_like(inner_val_data, t, alpha)
                    n_trials = sum(len(data[k]) for k in inner_val_data)
                    val_lls_inner.append(ll_val / n_trials)

                if skip_alpha:
                    break  # stop alpha loop for this t

                # Only add to dictionary if we have computed validation LLs
                if val_lls_inner:
                    mean_val_ll_per_hyper[(t, alpha)] = np.mean(val_lls_inner)
                # Select hyperparameters that maximize inner validation LL
                best_t, best_alpha = max(mean_val_ll_per_hyper, key=mean_val_ll_per_hyper.get)
                best_hyperparams_per_fold.append((best_t, best_alpha))

                # --- OUTER FOLD: evaluate predictive performance ---
                ll_test = OCS.get_total_log_like(outer_test_data, best_t, best_alpha)
                n_trials_test = sum(len(data[k]) for k in outer_test_data)
                outer_val_lls.append(ll_test / n_trials_test)

    mean_val_ll = np.mean(outer_val_lls)

    print(f"mean per-trial validation LL: {mean_val_ll:.4f}")

    return outer_val_lls, best_hyperparams_per_fold, mean_val_ll


In [ ]:
allData, D, lookupTree = OCS.precompute_possible_scores(participant_trials, item_space)

In [6]:
error_rate2 = np.mean([e['ERRORS']/4 for e in participant_trials if e['DEPTH'] == 2])
error_rate3 = np.mean([e['ERRORS']/4 for e in participant_trials if e['DEPTH'] == 3])

num2 = len([t for t in participant_trials if t['DEPTH'] == 2])
num3 = len(participant_trials) - num2
# for 2 level 
er2 = (3/2)*error_rate2
# for 3 level
er3 = (7/4)*error_rate3

print(f'Avg errors = {round(((er2*num2) + (er3)*num3)/len(participant_trials), 2)}')

Avg errors = 0.08


In [7]:
temps = np.linspace(1.0, 5, 41)
alphas = np.linspace(0, 0.5, 51)

soft_t, soft_a, soft_ll = OCS.find_best_params(allData, (temps, alphas), False)

# can't have alpha = 0 in deterministic model
det_a, det_ll = OCS.find_best_params(allData, (alphas[1:]), True)

In [8]:
print('CKMM')
print(f'\tSoftmax: {soft_ll}, T = {soft_t}, alpha = {soft_a}')
print(f'\tDeterministic: {det_ll}, alpha = {det_a}')

CKMM
	Softmax: -6103.1855845144255, T = 1.0, alpha = 0.15
	Deterministic: -6719.433185177522, alpha = 0.27


### Rational Models

In [9]:
item_values = (np.array(item_space) - min(item_space)) / (max(item_space) - min(item_space))
item_values = [np.array([val]) for val in item_values.tolist()]

data_rcm = format_rcm_data(participant_trials)
ll_rcm = get_total_log_like_rcm(data_rcm, item_values, 0.3, alpha=0.0, diff3=True)
print(ll_rcm)

-7302.143956858989


### Random assignment

In [136]:
num_3 = len([t for t in participant_trials if t['DEPTH'] == 3])
num_2 = len([t for t in participant_trials if t['DEPTH'] == 2])
 
random_2level = np.log(1/3)*13 # categorizing 4 distractors + 9 novel stimuli
random_3level = np.log(1/7)*13
random_likelihood = (random_2level*num_2) + (random_3level*num_3)
print(random_likelihood)

-13061.001257833486
